# This notebook contains the code for section 3.8


In [2]:
using DynamicPolynomials
using JuMP
using MosekTools
using LinearAlgebra
using CSV
using DataFrames
import JSON
using Dates
using Gurobi
using Symbolics

# Bell's Inequality (building towards CHSH)

In [4]:
using SumOfSquares

In [5]:
@polyvar pₕₕₕ pₕₕₜ pₕₜₕ pₕₜₜ pₜₕₕ pₜₕₜ pₜₜₕ pₜₜₜ
# @polyvar s_AB s_AC s_BC

vars =[pₕₕₕ pₕₕₜ pₕₜₕ pₕₜₜ pₜₕₕ pₜₕₜ pₜₜₕ pₜₜₜ]

1×8 Matrix{DynamicPolynomials.Variable{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}}}:
 pₕₕₕ  pₕₕₜ  pₕₜₕ  pₕₜₜ  pₜₕₕ  pₜₕₜ  pₜₜₕ  pₜₜₜ

In [6]:
# axioms other than non-negativity, then non-negativity ones
# axioms=[
#     pₕₕₕ+pₕₕₜ+pₕₜₕ+pₕₜₜ+pₜₕₕ+pₜₕₜ+pₜₜₕ+pₜₜₜ-1,
#     s_AB-pₕₕₕ-pₕₕₜ-pₜₜₕ-pₜₜₜ,
#     s_AC-pₕₕₕ-pₕₜₕ-pₜₕₜ-pₜₜₜ,
#     s_BC-pₕₕₕ-pₕₜₜ-pₜₕₜ-pₜₜₜ
#     ]

p_AB = pₕₕₕ+pₕₕₜ+pₜₜₕ+pₜₜₜ
p_AC = pₕₕₕ+pₕₜₕ+pₜₕₜ+pₜₜₜ
p_BC =pₕₕₕ+pₕₜₜ+pₜₕₕ+pₜₜₜ


S = @set pₕₕₕ >= 0 && pₕₕₜ >=0 && pₕₜₕ >=0 && pₕₜₜ >=0 && pₜₕₕ >=0 && pₜₕₜ >=0 && pₜₜₕ >=0 && pₜₜₜ >=0 && pₕₕₕ+pₕₕₜ+pₕₜₕ+pₕₜₜ+pₜₕₕ+pₜₕₜ+pₜₜₕ+pₜₜₜ==1

p= p_AB+p_AC+p_BC

3pₜₜₜ + pₜₜₕ + pₜₕₜ + pₜₕₕ + pₕₜₜ + pₕₜₕ + pₕₕₜ + 3pₕₕₕ

In [7]:
model= SOSModel(Mosek.Optimizer)

@variable(model, gamma)
@objective(model, Max, gamma)
con_ref=@constraint(model, p>=gamma, domain=S)
optimize!(model)

Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : LO (linear optimization problem)
  Constraints            : 8               
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 10              
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 0                 time                   : 0.00            
Lin. dep.  - primal attempts        : 0                 successes              : 0               
Lin. dep.  - dual attempts          : 0                 successes              : 0               
Lin. dep.  - primal deps.  

In [10]:
@show termination_status(model)
@show objective_value(model)

termination_status(model) = MathOptInterface.OPTIMAL
objective_value(model) = 1.0


1.0

In [11]:
@show value(gamma)

value(gamma) = 1.0


1.0

In [73]:
# Verifies that a valid lower bound on this inequality is 1.0
# Let us now extract the multipliers that verify this, to provide an axiomatic "proof" of the result


In [12]:
@show pₕₕₕ*SumOfSquares.lagrangian_multipliers(con_ref)[1][1]+pₕₕₜ * SumOfSquares.lagrangian_multipliers(con_ref)[2][1]+pₕₜₕ * SumOfSquares.lagrangian_multipliers(con_ref)[3][1]+ pₕₜₜ * SumOfSquares.lagrangian_multipliers(con_ref)[4][1]+ pₜₕₕ * SumOfSquares.lagrangian_multipliers(con_ref)[5][1]+ pₜₕₜ * SumOfSquares.lagrangian_multipliers(con_ref)[6][1]+ pₜₜₕ * SumOfSquares.lagrangian_multipliers(con_ref)[7][1]+ pₜₜₜ * SumOfSquares.lagrangian_multipliers(con_ref)[8][1]

pₕₕₕ * ((SumOfSquares.lagrangian_multipliers(con_ref))[1])[1] + pₕₕₜ * ((SumOfSquares.lagrangian_multipliers(con_ref))[2])[1] + pₕₜₕ * ((SumOfSquares.lagrangian_multipliers(con_ref))[3])[1] + pₕₜₜ * ((SumOfSquares.lagrangian_multipliers(con_ref))[4])[1] + pₜₕₕ * ((SumOfSquares.lagrangian_multipliers(con_ref))[5])[1] + pₜₕₜ * ((SumOfSquares.lagrangian_multipliers(con_ref))[6])[1] + pₜₜₕ * ((SumOfSquares.lagrangian_multipliers(con_ref))[7])[1] + pₜₜₜ * ((SumOfSquares.lagrangian_multipliers(con_ref))[8])[1] = 2.0pₜₜₜ + 2.0pₕₕₕ


2.0pₜₜₜ + 2.0pₕₕₕ

This verifies that P_ab+P_ac+P_bc>=1, since this is 2.0 p_hh+2.0 p_tt>=0 mod ideal.

# CHSH inequality now

Define random variables A, A', B, B' which take values +-1

Let s=AB-AB'+A'B+A'B'

We want to show that E[|S|]<=2, which violates the 2 sqrt(2) value predicted from Trirelson's bound.

In [13]:
@polyvar pₕₕₕₕ pₕₕₜₕ pₕₜₕₕ pₕₜₜₕ pₜₕₕₕ pₜₕₜₕ pₜₜₕₕ pₜₜₜₕ pₕₕₕₜ pₕₕₜₜ pₕₜₕₜ pₕₜₜₜ pₜₕₕₜ pₜₕₜₜ pₜₜₕₜ pₜₜₜₜ

S=@set pₕₕₕₕ >=0.0 && pₕₕₜₕ >=0.0 && pₕₜₕₕ >=0.0 && pₕₜₜₕ >=0.0 && pₜₕₕₕ >=0.0 && pₜₕₜₕ >=0.0 && pₜₜₕₕ >=0.0 && pₜₜₜₕ >=0.0 && pₕₕₕₜ >= 0.0 && pₕₕₜₜ >= 0.0 && pₕₜₕₜ >=0.0 && pₕₜₜₜ >= 0.0 && pₜₕₕₜ >=0.0 &&  pₜₕₜₜ >= 0.0 && pₜₜₕₜ >= 0.0 && pₜₜₜₜ >= 0.0&& pₕₕₕₕ + pₕₕₜₕ + pₕₜₕₕ + pₕₜₜₕ + pₜₕₕₕ + pₜₕₜₕ + pₜₜₕₕ + pₜₜₜₕ + pₕₕₕₜ + pₕₕₜₜ + pₕₜₕₜ + pₕₜₜₜ + pₜₕₕₜ + pₜₕₜₜ + pₜₜₕₜ + pₜₜₜₜ== 1.0

E_abs_s=2.0*(pₕₕₕₕ + pₕₕₜₕ + pₕₜₕₕ + pₕₜₜₕ + pₜₕₕₕ + pₜₕₜₕ + pₜₜₕₕ + pₜₜₜₕ + pₕₕₕₜ + pₕₕₜₜ + pₕₜₕₜ + pₕₜₜₜ + pₜₕₕₜ + pₜₕₜₜ + pₜₜₕₜ + pₜₜₜₜ)



2.0pₜₜₜₜ + 2.0pₜₜₕₜ + 2.0pₜₕₜₜ + 2.0pₜₕₕₜ + 2.0pₕₜₜₜ + 2.0pₕₜₕₜ + 2.0pₕₕₜₜ + 2.0pₕₕₕₜ + 2.0pₜₜₜₕ + 2.0pₜₜₕₕ + 2.0pₜₕₜₕ + 2.0pₜₕₕₕ + 2.0pₕₜₜₕ + 2.0pₕₜₕₕ + 2.0pₕₕₜₕ + 2.0pₕₕₕₕ

In [14]:
model= SOSModel(Mosek.Optimizer)

@variable(model, gamma)
@objective(model, Min, gamma)
con_ref=@constraint(model, gamma>=E_abs_s, domain=S)
optimize!(model)

Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : LO (linear optimization problem)
  Constraints            : 16              
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 18              
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Eliminator - tries                  : 0                 time                   : 0.00            
Lin. dep.  - tries                  : 0                 time                   : 0.00            
Lin. dep.  - primal attempts        : 0                 successes              : 0               
Lin. dep.  - dual attempts          : 0                 successes              : 0               
Lin. dep.  - primal deps.           : 0                 dual deps.             : 0               
Presolve

In [15]:
@show objective_value(model)

objective_value(model) = 2.0


2.0

In [16]:
@show lagrangian_multipliers(con_ref)

lagrangian_multipliers(con_ref) = GramMatrix{Float64, MonomialBasis{Monomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}}, MonomialVector{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}}}, Float64, SymMatrix{Float64}}[GramMatrix with row/column basis:
 MonomialBasis([1])
And entries in a 1×1 SymMatrix{Float64}:
 0.0, GramMatrix with row/column basis:
 MonomialBasis([1])
And entries in a 1×1 SymMatrix{Float64}:
 -0.0, GramMatrix with row/column basis:
 MonomialBasis([1])
And entries in a 1×1 SymMatrix{Float64}:
 -0.0, GramMatrix with row/column basis:
 MonomialBasis([1])
And entries in a 1×1 SymMatrix{Float64}:
 -0.0, GramMatrix with row/column basis:
 MonomialBasis([1])
And entries in a 1×1 SymMatrix{Float64}:
 -0.0, GramMatrix with row/column basis:
 MonomialBasis([1])
And entries in a 1×1 SymMatrix{Float64}:
 -0.0, GramMatrix with row/column basis:
 MonomialBasis([1])
And entries in a 1×1 SymMatrix{Float64}:
 -

16-element Vector{GramMatrix{Float64, MonomialBasis{Monomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}}, MonomialVector{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}}}, Float64, SymMatrix{Float64}}}:
 GramMatrix with row/column basis:
 MonomialBasis([1])
And entries in a 1×1 SymMatrix{Float64}:
 0.0
 GramMatrix with row/column basis:
 MonomialBasis([1])
And entries in a 1×1 SymMatrix{Float64}:
 -0.0
 GramMatrix with row/column basis:
 MonomialBasis([1])
And entries in a 1×1 SymMatrix{Float64}:
 -0.0
 GramMatrix with row/column basis:
 MonomialBasis([1])
And entries in a 1×1 SymMatrix{Float64}:
 -0.0
 GramMatrix with row/column basis:
 MonomialBasis([1])
And entries in a 1×1 SymMatrix{Float64}:
 -0.0
 GramMatrix with row/column basis:
 MonomialBasis([1])
And entries in a 1×1 SymMatrix{Float64}:
 -0.0
 GramMatrix with row/column basis:
 MonomialBasis([1])
And entries in a 1×1 SymMatrix{Float64}:
 -0.0
 GramMatr

# Multipliers are all zero, which is because this inequality is 0.0>=0.0 mod ideal.

In [17]:
# Deriving the I3322 inequality, using indexing.
# Let "1" denote -1, "2" denote 1
@polyvar p[1:2, 1:2, 1:2, 1:2, 1:2, 1:2]
S=@set sum(p)==1 
# Append to set, without writing out fully (nested loop since syntax to do more nicely does not, to my knowledge, currently exist)
for i1=1:2
    for i2=1:2
        for i3=1:2
            for i4=1:2
                for i5=1:2
                    for i6=1:2
                        S=@set S && p[i1,i2,i3,i4,i5,i6]>=0.0
                    end
                end
            end
        end
    end
end

In [18]:
E_A1=-1.0*sum(p[1,:,:,:,:,:])+1.0*sum(p[2,:,:,:,:,:])
E_A2=-1.0*sum(p[:,1,:,:,:,:])+1.0*sum(p[:,2,:,:,:,:])

E_B1=-1.0*sum(p[:,:,:,1,:,:])+1.0*sum(p[:,:,:,2,:,:])
E_B2=-1.0*sum(p[:,:,:,:,1,:])+1.0*sum(p[:,:,:,:,2,:])

E_A1B1=1.0*sum(p[1,:,:,1,:,:])-1.0*sum(p[1,:,:,2,:,:])-1.0*sum(p[2,:,:,1,:,:])+1.0*sum(p[2,:,:,2,:,:])
E_A1B2=1.0*sum(p[1,:,:,:,1,:])-1.0*sum(p[1,:,:,:,2,:])-1.0*sum(p[2,:,:,:,1,:])+1.0*sum(p[2,:,:,:,2,:])
E_A2B1=1.0*sum(p[:,1,:,1,:,:])-1.0*sum(p[:,1,:,2,:,:])-1.0*sum(p[:,2,:,1,:,:])+1.0*sum(p[:,2,:,2,:,:])
E_A2B2=1.0*sum(p[:,1,:,:,1,:])-1.0*sum(p[:,1,:,:,2,:])-1.0*sum(p[:,2,:,:,1,:])+1.0*sum(p[:,2,:,:,2,:])
E_A1B3=1.0*sum(p[1,:,:,:,:,1])-1.0*sum(p[1,:,:,:,:,2])-1.0*sum(p[2,:,:,:,:,1])+1.0*sum(p[2,:,:,:,:,2])
E_A2B3=1.0*sum(p[:,1,:,:,:,1])-1.0*sum(p[:,1,:,:,:,2])-1.0*sum(p[:,2,:,:,:,1])+1.0*sum(p[:,2,:,:,:,2])
E_A3B1=1.0*sum(p[:,:,1,1,:,:])-1.0*sum(p[:,:,1,2,:,:])-1.0*sum(p[:,:,2,1,:,:])+1.0*sum(p[:,:,2,2,:,:])
E_A3B2=1.0*sum(p[:,:,1,:,1,:])-1.0*sum(p[:,:,1,:,2,:])-1.0*sum(p[:,:,2,:,1,:])+1.0*sum(p[:,:,2,:,2,:])



E_int=E_A1B1+E_A2B2-E_A1B2-E_A2B1
E_int2=E_A1B3+E_A2B3
E_int3=E_A3B1+E_A3B2

E_overall=E_A1-E_A2+E_B1-E_B2-E_int+E_int2+E_int3

4.0p₂₋₂₋₂₋₂₋₂₋₂ + 4.0p₂₋₁₋₂₋₂₋₂₋₂ - 4.0p₁₋₂₋₁₋₂₋₂₋₂ - 4.0p₁₋₁₋₁₋₂₋₂₋₂ - 8.0p₁₋₂₋₂₋₁₋₂₋₂ + 4.0p₂₋₁₋₂₋₁₋₂₋₂ - 4.0p₁₋₁₋₂₋₁₋₂₋₂ - 8.0p₁₋₂₋₁₋₁₋₂₋₂ + 4.0p₂₋₁₋₁₋₁₋₂₋₂ - 4.0p₁₋₁₋₁₋₁₋₂₋₂ + 4.0p₂₋₂₋₂₋₂₋₁₋₂ + 4.0p₁₋₂₋₂₋₂₋₁₋₂ + 4.0p₂₋₂₋₁₋₂₋₁₋₂ + 4.0p₁₋₂₋₁₋₂₋₁₋₂ - 4.0p₁₋₂₋₂₋₁₋₁₋₂ - 4.0p₁₋₁₋₂₋₁₋₁₋₂ + 4.0p₂₋₂₋₁₋₁₋₁₋₂ + 4.0p₂₋₁₋₁₋₁₋₁₋₂ + 4.0p₂₋₁₋₂₋₂₋₂₋₁ + 4.0p₁₋₁₋₂₋₂₋₂₋₁ - 4.0p₂₋₂₋₁₋₂₋₂₋₁ - 4.0p₁₋₂₋₁₋₂₋₂₋₁ - 4.0p₂₋₂₋₂₋₁₋₂₋₁ - 8.0p₁₋₂₋₂₋₁₋₂₋₁ + 4.0p₂₋₁₋₂₋₁₋₂₋₁ - 4.0p₂₋₂₋₁₋₁₋₂₋₁ - 8.0p₁₋₂₋₁₋₁₋₂₋₁ + 4.0p₂₋₁₋₁₋₁₋₂₋₁ + 4.0p₁₋₂₋₂₋₂₋₁₋₁ + 4.0p₁₋₁₋₂₋₂₋₁₋₁ + 4.0p₁₋₂₋₁₋₂₋₁₋₁ + 4.0p₁₋₁₋₁₋₂₋₁₋₁ - 4.0p₂₋₂₋₂₋₁₋₁₋₁ - 4.0p₁₋₂₋₂₋₁₋₁₋₁ + 4.0p₂₋₁₋₁₋₁₋₁₋₁ + 4.0p₁₋₁₋₁₋₁₋₁₋₁

In [19]:
model= SOSModel(Mosek.Optimizer)

@variable(model, gamma)
@objective(model, Min, gamma)
con_ref=@constraint(model, gamma>=E_overall, domain=S)
optimize!(model)

Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : LO (linear optimization problem)
  Constraints            : 64              
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 66              
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Eliminator - tries                  : 0                 time                   : 0.00            
Lin. dep.  - tries                  : 0                 time                   : 0.00            
Lin. dep.  - primal attempts        : 0                 successes              : 0               
Lin. dep.  - dual attempts          : 0                 successes              : 0               
Lin. dep.  - primal deps.           : 0                 dual deps.             : 0               
Presolve

In [20]:
@show objective_value(model)

objective_value(model) = 4.0


4.0

In [21]:
@show sum((vec(p).*lagrangian_multipliers(con_ref)))

sum(vec(p) .* lagrangian_multipliers(con_ref)) = 4.0p₁₋₂₋₂₋₂₋₂₋₂ + 4.0p₁₋₁₋₂₋₂₋₂₋₂ + 4.0p₂₋₂₋₁₋₂₋₂₋₂ + 8.0p₁₋₂₋₁₋₂₋₂₋₂ + 4.0p₂₋₁₋₁₋₂₋₂₋₂ + 8.0p₁₋₁₋₁₋₂₋₂₋₂ + 4.0p₂₋₂₋₂₋₁₋₂₋₂ + 8.0p₁₋₂₋₂₋₁₋₂₋₂ + 4.0p₁₋₁₋₂₋₁₋₂₋₂ + 4.0p₂₋₂₋₁₋₁₋₂₋₂ + 8.0p₁₋₂₋₁₋₁₋₂₋₂ + 4.0p₁₋₁₋₁₋₁₋₂₋₂ + 4.0p₂₋₁₋₂₋₂₋₁₋₂ + 4.0p₁₋₁₋₂₋₂₋₁₋₂ + 4.0p₂₋₁₋₁₋₂₋₁₋₂ + 4.0p₁₋₁₋₁₋₂₋₁₋₂ + 4.0p₂₋₂₋₂₋₁₋₁₋₂ + 4.0p₁₋₂₋₂₋₁₋₁₋₂ + 4.0p₂₋₁₋₂₋₁₋₁₋₂ + 4.0p₁₋₁₋₂₋₁₋₁₋₂ + 4.0p₂₋₂₋₂₋₂₋₂₋₁ + 4.0p₁₋₂₋₂₋₂₋₂₋₁ + 12.0p₂₋₂₋₁₋₂₋₂₋₁ + 12.0p₁₋₂₋₁₋₂₋₂₋₁ + 8.0p₂₋₁₋₁₋₂₋₂₋₁ + 8.0p₁₋₁₋₁₋₂₋₂₋₁ + 8.0p₂₋₂₋₂₋₁₋₂₋₁ + 8.0p₁₋₂₋₂₋₁₋₂₋₁ + 12.0p₂₋₂₋₁₋₁₋₂₋₁ + 12.0p₁₋₂₋₁₋₁₋₂₋₁ + 4.0p₂₋₁₋₁₋₁₋₂₋₁ + 4.0p₁₋₁₋₁₋₁₋₂₋₁ + 4.0p₂₋₂₋₂₋₂₋₁₋₁ + 4.0p₂₋₁₋₂₋₂₋₁₋₁ + 8.0p₂₋₂₋₁₋₂₋₁₋₁ + 4.0p₁₋₂₋₁₋₂₋₁₋₁ + 8.0p₂₋₁₋₁₋₂₋₁₋₁ + 4.0p₁₋₁₋₁₋₂₋₁₋₁ + 8.0p₂₋₂₋₂₋₁₋₁₋₁ + 4.0p₁₋₂₋₂₋₁₋₁₋₁ + 4.0p₂₋₁₋₂₋₁₋₁₋₁ + 8.0p₂₋₂₋₁₋₁₋₁₋₁ + 4.0p₁₋₂₋₁₋₁₋₁₋₁ + 4.0p₂₋₁₋₁₋₁₋₁₋₁


4.0p₁₋₂₋₂₋₂₋₂₋₂ + 4.0p₁₋₁₋₂₋₂₋₂₋₂ + 4.0p₂₋₂₋₁₋₂₋₂₋₂ + 8.0p₁₋₂₋₁₋₂₋₂₋₂ + 4.0p₂₋₁₋₁₋₂₋₂₋₂ + 8.0p₁₋₁₋₁₋₂₋₂₋₂ + 4.0p₂₋₂₋₂₋₁₋₂₋₂ + 8.0p₁₋₂₋₂₋₁₋₂₋₂ + 4.0p₁₋₁₋₂₋₁₋₂₋₂ + 4.0p₂₋₂₋₁₋₁₋₂₋₂ + 8.0p₁₋₂₋₁₋₁₋₂₋₂ + 4.0p₁₋₁₋₁₋₁₋₂₋₂ + 4.0p₂₋₁₋₂₋₂₋₁₋₂ + 4.0p₁₋₁₋₂₋₂₋₁₋₂ + 4.0p₂₋₁₋₁₋₂₋₁₋₂ + 4.0p₁₋₁₋₁₋₂₋₁₋₂ + 4.0p₂₋₂₋₂₋₁₋₁₋₂ + 4.0p₁₋₂₋₂₋₁₋₁₋₂ + 4.0p₂₋₁₋₂₋₁₋₁₋₂ + 4.0p₁₋₁₋₂₋₁₋₁₋₂ + 4.0p₂₋₂₋₂₋₂₋₂₋₁ + 4.0p₁₋₂₋₂₋₂₋₂₋₁ + 12.0p₂₋₂₋₁₋₂₋₂₋₁ + 12.0p₁₋₂₋₁₋₂₋₂₋₁ + 8.0p₂₋₁₋₁₋₂₋₂₋₁ + 8.0p₁₋₁₋₁₋₂₋₂₋₁ + 8.0p₂₋₂₋₂₋₁₋₂₋₁ + 8.0p₁₋₂₋₂₋₁₋₂₋₁ + 12.0p₂₋₂₋₁₋₁₋₂₋₁ + 12.0p₁₋₂₋₁₋₁₋₂₋₁ + 4.0p₂₋₁₋₁₋₁₋₂₋₁ + 4.0p₁₋₁₋₁₋₁₋₂₋₁ + 4.0p₂₋₂₋₂₋₂₋₁₋₁ + 4.0p₂₋₁₋₂₋₂₋₁₋₁ + 8.0p₂₋₂₋₁₋₂₋₁₋₁ + 4.0p₁₋₂₋₁₋₂₋₁₋₁ + 8.0p₂₋₁₋₁₋₂₋₁₋₁ + 4.0p₁₋₁₋₁₋₂₋₁₋₁ + 8.0p₂₋₂₋₂₋₁₋₁₋₁ + 4.0p₁₋₂₋₂₋₁₋₁₋₁ + 4.0p₂₋₁₋₂₋₁₋₁₋₁ + 8.0p₂₋₂₋₁₋₁₋₁₋₁ + 4.0p₁₋₂₋₁₋₁₋₁₋₁ + 4.0p₂₋₁₋₁₋₁₋₁₋₁